In [10]:
import os
import copy
import torch
import detectron2
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_test_loader, build_detection_train_loader

In [11]:
# # Register Dataset
# try: # register_coco_instances 함수를 사용해 COCO 형식의 데이터셋을 등록
#     register_coco_instances('coco_trash_train', {}, '../../dataset/train.json', '../../dataset/')
# except AssertionError:
#     pass

# try: # 
#     register_coco_instances('coco_trash_test', {}, '../../dataset/test.json', '../../dataset/')
# except AssertionError:
#     pass

# # MetadataCatalog.get()를 통해 coco_trash_train 데이터셋의 클래스 이름을 지정
# MetadataCatalog.get('coco_trash_train').thing_classes = ["General trash", "Paper", "Paper pack", "Metal", 
#                                                          "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

In [12]:
import os
import json
import random
import numpy as np
from sklearn.model_selection import StratifiedKFold
from detectron2.data.datasets import register_coco_instances

# 절대 경로를 사용하여 데이터셋 경로 설정
dataset_dir = '/data/ephemeral/home/Lv2.Object_Detection/dataset'
train_json_path = os.path.join(dataset_dir, 'train.json')
image_dir = os.path.join(dataset_dir,)  # 실제 이미지 경로로 수정

# COCO 형식의 train.json 로드
with open(train_json_path, 'r') as f:
    coco_data = json.load(f)

# image_id 별로 annotations를 묶기
image_to_annotations = {}
image_to_category = {}  # StratifiedKFold를 사용하기 위해 클래스 레이블 필요
for anno in coco_data['annotations']:
    image_id = anno['image_id']
    if image_id not in image_to_annotations:
        image_to_annotations[image_id] = []
    image_to_annotations[image_id].append(anno)
    # 이미지에 속한 클래스 레이블 추가 (첫 번째 어노테이션 기준으로 레이블 설정)
    if image_id not in image_to_category:
        image_to_category[image_id] = anno['category_id']

# 이미지 리스트 및 해당하는 클래스 라벨 추출
image_ids = list(image_to_annotations.keys())
image_labels = [image_to_category[image_id] for image_id in image_ids]

# StratifiedKFold 설정
n_splits = 5  # 원하는 K 값을 설정
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 원하는 fold 선택 (예: fold_idx = 0일 경우 첫 번째 fold를 validation으로 사용)
fold_idx = 0

# K-fold split 진행
for idx, (train_idx, val_idx) in enumerate(skf.split(image_ids, image_labels)):
    if idx == fold_idx:
        train_image_ids = [image_ids[i] for i in train_idx]
        val_image_ids = [image_ids[i] for i in val_idx]
        break

# Train과 Val에 해당하는 annotations 필터링
train_annotations = [anno for image_id in train_image_ids for anno in image_to_annotations[image_id]]
val_annotations = [anno for image_id in val_image_ids for anno in image_to_annotations[image_id]]

# Train과 Val에 해당하는 이미지 필터링
train_images = [img for img in coco_data['images'] if img['id'] in train_image_ids]
val_images = [img for img in coco_data['images'] if img['id'] in val_image_ids]

# train.json과 val.json 생성
train_split_path = os.path.join(dataset_dir, f'train_fold_{fold_idx}.json')
val_split_path = os.path.join(dataset_dir, f'val_fold_{fold_idx}.json')

train_data = coco_data.copy()
train_data['annotations'] = train_annotations
train_data['images'] = train_images
with open(train_split_path, 'w') as f:
    json.dump(train_data, f)

val_data = coco_data.copy()
val_data['annotations'] = val_annotations
val_data['images'] = val_images
with open(val_split_path, 'w') as f:
    json.dump(val_data, f)

# 데이터셋 등록
register_coco_instances(f"coco_trash_train_fold_{fold_idx}", {}, train_split_path, image_dir)
register_coco_instances(f"coco_trash_val_fold_{fold_idx}", {}, val_split_path, image_dir)

In [13]:
# config 불러오기
'''
1. get_cfg()를 호출해 기본 설정을 가져오기

2. model_zoo.get_config_file()을 사용해 미리 정의된 Faster R-CNN의 R101 FPN 3x 구성 파일을 로드(이 부분은 변경 가능)

'''



cfg = get_cfg() # detectron2에서 기본 설정을 가지고 오는 함수입니다.
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml'))

In [14]:
# 원하는 폴드 인덱스 설정
fold_idx = 0  # 0부터 4까지 가능, 사용하고자 하는 폴드를 설정

# K-fold로 나눈 데이터를 기반으로, 학습 및 검증 데이터셋 설정
cfg.DATASETS.TRAIN = (f'coco_trash_train_fold_{fold_idx}',)
cfg.DATASETS.TEST = (f'coco_trash_val_fold_{fold_idx}',)

# DataLoader에서 사용할 worker 수를 2로 설정 (병렬 데이터 로딩)
cfg.DATALOADER.NUM_WORKERS = 2  # (오타 수정: NUM_WOREKRS → NUM_WORKERS)

# # Faster R-CNN R101 FPN 3x 모델의 사전 학습된 가중치 사용
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml')

# 한 번의 학습 배치에서 처리할 이미지 수를 4로 설정
cfg.SOLVER.IMS_PER_BATCH = 4

# 학습률(Learning Rate)을 0.001로 설정
cfg.SOLVER.BASE_LR = 0.001

# 학습 반복(iteration)을 최대 15,000번으로 설정
cfg.SOLVER.MAX_ITER = 15000

# 8000번째와 12000번째 반복(iteration)에서 학습률을 감소시키도록 설정
cfg.SOLVER.STEPS = (8000, 12000)

# 학습률 감소 비율을 0.005로 설정
cfg.SOLVER.GAMMA = 0.005

# 체크포인트 저장 주기를 3000번 반복마다 저장하도록 설정
cfg.SOLVER.CHECKPOINT_PERIOD = 3000

# 모델의 출력(결과) 파일을 저장할 디렉토리를 './output'으로 설정
cfg.OUTPUT_DIR = './output/NMS(0dot5)'

# 이미지당 ROI(Region of Interest) 샘플 수를 128로 설정 (RoI Head의 배치 크기)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128

# 모델의 클래스 수를 10개로 설정 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10

# 평가 주기를 3000번 반복마다 평가하도록 설정 (TEST 단계)
cfg.TEST.EVAL_PERIOD = 3000

In [15]:
# mapper - input data를 어떤 형식으로 return할지 (따라서 augmnentation 등 데이터 전처리 포함 됨)
'''
데이터 매퍼 (전처리) 설정:

MyMapper 함수는 입력 데이터에 대한 전처리 방법을 정의

이미지에 랜덤으로 수직 뒤집기, 밝기 및 대비 변환을 적용

변환된 이미지를 텐서로 변환하고 어노테이션을 조정하여 dataset_dict에 추가

'''

import detectron2.data.transforms as T

def MyMapper(dataset_dict):
    # 원본 데이터 복사하여 데이터 변형 시 원본 데이터가 손상되지 않도록 함
    dataset_dict = copy.deepcopy(dataset_dict)
    
    # 이미지를 'BGR' 형식으로 불러옴 (Detectron2의 기본 설정은 BGR임)
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    
    # 데이터 증강(transform) 리스트 설정
    transform_list = [
        T.RandomFlip(prob=0.5, horizontal=False, vertical=True),  # 50% 확률로 이미지를 수직으로 뒤집음
        T.RandomBrightness(0.8, 1.8),  # 이미지 밝기를 랜덤으로 조정 (0.8배 ~ 1.8배)
        T.RandomContrast(0.6, 1.3)  # 이미지 대비를 랜덤으로 조정 (0.6배 ~ 1.3배)
    ]
    
    # 설정한 transform 리스트를 적용하여 이미지를 변환
    image, transforms = T.apply_transform_gens(transform_list, image)
    
    # 변환된 이미지를 텐서(tensor) 형식으로 변환하여 dataset_dict에 저장 (Detectron2의 입력 형식에 맞춤)
    dataset_dict['image'] = torch.as_tensor(image.transpose(2,0,1).astype('float32'))
    
    # 어노테이션(annotations)을 변환된 이미지에 맞춰 적용 (변형된 이미지 좌표계에 맞게 재조정)
    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop('annotations')  # 'annotations'에서 하나씩 가져와 변환 수행
        if obj.get('iscrowd', 0) == 0  # 'iscrowd'가 0인 객체만 선택 (crowd 객체 제외)
    ]
    
    # 변환된 어노테이션을 바탕으로 'instances' 생성 (Detectron2에서 인스턴스 예측을 위한 포맷)
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    
    # 유효하지 않은 인스턴스(빈 인스턴스)를 필터링하여 제거
    dataset_dict['instances'] = utils.filter_empty_instances(instances)
    
    # 최종적으로 변형된 dataset_dict 반환
    return dataset_dict

In [16]:
# trainer - DefaultTrainer를 상속
class MyTrainer(DefaultTrainer):
    
    @classmethod
    def build_train_loader(cls, cfg, sampler=None):
        return build_detection_train_loader(
        cfg, mapper = MyMapper, sampler = sampler
        )
    
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs('./output_eval', exist_ok = True)
            output_folder = './output_eval'
            
        return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [17]:
import os
import wandb
from detectron2.engine import DefaultTrainer

# wandb 설정
wandb.init(project="my-detectron2-project", config=cfg, name="experiment_name")

# 필요한 경우 cfg 값 로그
wandb.config.update(cfg)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [18]:
# train
os.makedirs(cfg.OUTPUT_DIR, exist_ok = True)

trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[10/11 14:05:01 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (11, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (11,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (40, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (40,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias

[10/11 14:05:03 d2.engine.train_loop]: Starting training from iteration 0
ERROR [10/11 14:05:07 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "/data/ephemeral/home/Lv2.Object_Detection/baseline/detectron2/detectron2/engine/train_loop.py", line 155, in train
    self.run_step()
  File "/data/ephemeral/home/Lv2.Object_Detection/baseline/detectron2/detectron2/engine/defaults.py", line 530, in run_step
    self._trainer.run_step()
  File "/data/ephemeral/home/Lv2.Object_Detection/baseline/detectron2/detectron2/engine/train_loop.py", line 310, in run_step
    loss_dict = self.model(data)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/data/ephemeral/home/Lv2.Object_Detection/baseline/detectron2/detectron2/modeling/meta_arch/rcnn.py", line 172, in forward
    features = self.backbone(images.tensor)
  File "/opt/conda/lib/python3.10/site-packag

RuntimeError: CUDA out of memory. Tried to allocate 256.00 MiB (GPU 0; 31.74 GiB total capacity; 3.07 GiB already allocated; 244.38 MiB free; 3.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# 학습 종료 시 로그
wandb.finish()